In [6]:
!pip install piper-tts

In [7]:
!python3 -m piper.download_voices ru_RU-denis-medium ru_RU-dmitri-medium ru_RU-irina-medium ru_RU-ruslan-medium

INFO:__main__:Downloaded: ru_RU-denis-medium
INFO:__main__:Downloaded: ru_RU-dmitri-medium
INFO:__main__:Downloaded: ru_RU-irina-medium
INFO:__main__:Downloaded: ru_RU-ruslan-medium


In [8]:
import csv
import re
import os
import wave
from pathlib import Path
from piper import PiperVoice

In [9]:
CSV_PATH = "/content/drive/MyDrive/DATA/processed_dataset.csv"
TEXT_COL = "Desc"
OUT_DIR = Path("audio_out")
OUT_DIR.mkdir(exist_ok=True)

In [10]:
# --- Функция для безопасных имён файлов ---
def slug(text, n=40):
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r"[^0-9A-Za-zА-Яа-я _-]", "", text)[:n]
    return (text or "utt").replace(" ", "_")


In [11]:
# === Настройка голосов и шаг переключения ===
VOICE_FILES = [
    "/content/ru_RU-denis-medium.onnx",
    "/content/ru_RU-dmitri-medium.onnx",
    "/content/ru_RU-irina-medium.onnx",
    "/content/ru_RU-ruslan-medium.onnx",
]
SWITCH_EVERY = 50  # каждые 50 сообщений

# Предзагрузка голосов в память (быстрое переключение)
voices = [PiperVoice.load(vf) for vf in VOICE_FILES]
print(f"Загружено голосов: {len(voices)}")

def pick_voice(processed_count: int):
    """Вернёт PiperVoice для текущего обработанного номера (0..N-1)."""
    idx = (processed_count // SWITCH_EVERY) % len(voices)
    return voices[idx]


Загружено голосов: 4


In [12]:
processed = 0  # сколько реально озвучили (не путать с i из enumerate)

with open(CSV_PATH, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)  # или csv.reader, как у тебя
    for i, row in enumerate(reader):
        text = (row.get(TEXT_COL) or "").strip()
        if not text:
            continue

        # выбрать голос по счётчику processed
        voice = pick_voice(processed)

        filename = f"{i:08d}_{slug(text)}.wav"
        out_path = OUT_DIR / filename
        with wave.open(str(out_path), "wb") as wav_file:
            voice.synthesize_wav(text, wav_file)

        processed += 1
        if processed % 50 == 0:
            print(f"[{processed}] переключение голоса → {(processed//SWITCH_EVERY)%len(voices)}")
        if processed % 1000 == 0:
            print(f"[i={i}] {out_path} — готово")

print("Генерация завершена")


[50] переключение голоса → 1
[100] переключение голоса → 2
[150] переключение голоса → 3
[200] переключение голоса → 0
[250] переключение голоса → 1


Error: # channels not specified